In [10]:
import pandas as pd
import spot
import os
import re

In [11]:
test = False

In [12]:
telatko_current = '/home/ter/telatko/telatko.py'

In [13]:
telatko_prev_version = '/home/ter/Desktop/telatko-previous_paper_version/telatko.py'

In [14]:
automata_sets = ['delag', 'rabinizer4', 'ltl3tela', 'ltl2tgba']

In [15]:
if test:
    automata_sets = ['test_set']

In [16]:
timeout =15

In [17]:
os.environ['SPOT_TMPDIR'] = '/home/ter/telatko/september22_experiments/autcross'
#os.environ['SPOT_TMPKEEP'] = '1'

In [18]:
def run_autcross_SCC_UNOPT(tool):
    !autcross --no-checks --csv=patterns-telatko-SCC_UNOPT-{tool}.csv -F patterns-{tool}.hoa 'autfilt %H > %O' '{telatko_current} -F %H -T {timeout} -L 1 -I -O %O' '{telatko_current} -F %H -T {timeout} -L 2 -I -O %O' '{telatko_current} -F %H -T {timeout} -L 3 -I -O %O' '{telatko_current} -F %H -T {timeout} -I -O %O' > autcross/{tool}_SCC_UNOPT.log 2>&1

In [19]:
def run_autcross_SCC_OPT(tool):
    !autcross --no-checks --csv=patterns-telatko-SCC_OPT-{tool}.csv -F patterns-{tool}.hoa 'autfilt %H > %O' '{telatko_current} -F %H -T {timeout} -L 1 -I -S -O %O' '{telatko_current} -F %H -T {timeout} -L 2 -I -S -O %O' '{telatko_current} -F %H -T {timeout} -L 3 -I -S -O %O' '{telatko_current} -F %H -T {timeout} -I -S -O %O' > autcross/{tool}_SCC_OPT.log 2>&1

In [20]:
def run_autcross_PREV_VERSION(tool):
    !autcross --no-checks --csv=patterns-telatko-PREV_VERSION-{tool}.csv -F patterns-{tool}.hoa 'autfilt %H > %O' '{telatko_prev_version} -F %H -T {timeout} -L 1 -I -O %O' '{telatko_prev_version} -F %H -T {timeout} -L 2 -I -O %O' '{telatko_prev_version} -F %H -T {timeout} -L 3 -I -O %O' '{telatko_prev_version} -F %H -T {timeout} -I -O %O' > autcross/{tool}_PREV_VERSION.log 2>&1

In [21]:
for tool in automata_sets:
    print(tool)
    run_autcross_SCC_UNOPT(tool)
    print("SCC UNOPT done")
    run_autcross_SCC_OPT(tool)
    print("SCC OPT done")
    run_autcross_PREV_VERSION(tool)
    print("Prev version done")
    print(tool, " done")
    print("----------------------------")

delag
SCC UNOPT done
SCC OPT done
Prev version done
delag  done
----------------------------
rabinizer4
SCC UNOPT done
SCC OPT done
Prev version done
rabinizer4  done
----------------------------
ltl3tela
SCC UNOPT done
SCC OPT done
Prev version done
ltl3tela  done
----------------------------
ltl2tgba
SCC UNOPT done
SCC OPT done
Prev version done
ltl2tgba  done
----------------------------


In [22]:
def get_data(version):
    data = {}

    for tool in automata_sets:
        #print(tool)
        csv = pd.read_csv('patterns-telatko-'+version + '-' + tool + '.csv')

        header = {
            'L0.acc': [],
            'L1.acc': [],
            'L1.time': [],
            'L2.acc': [],
            'L2.time': [],
            'L3.acc': [],
            'L3.time': [],
            'L4.acc': [],
            'L4.time': [],
        }
        i = 0
        for _,r in csv.iterrows():
            #print(i)
            #print(r)
            #header[metrics[i]].append(r['output.acc_sets'])
            header['L' + str(i) + '.acc'].append(r['output.acc_sets'])
            if i > 0:
                header['L' + str(i) + '.time'].append(r['time'])
            i = (i + 1) % 5



        data_dd = pd.DataFrame(data = header)

        data_de = data_dd.dropna()
        #print(data_de)


        data_df = data_de[(data_de['L0.acc'] > 1)]
        data[tool] = data_df
        
        #print(data_df)
    return data



    

In [23]:
sumdata = {
    'tool': [],
    'autcount': [],
    'L0.acc': [],
}
for l in range(1, 5):
    for s in ['acc', 'time', 'ratio']:
        sumdata['L' + str(l) + '.' + s] = []

for tool in automata_sets:
    data_df = data[tool]
    
    sumdata['tool'].append(tool)
    sd = data_df.sum()
    sumdata['autcount'].append(len(data_df))
    for i in range(0, 5):
        sumdata['L' + str(i) + '.acc'].append(int(sd['L' + str(i) + '.acc']))
        if i > 0:
            t = sd['L' + str(i) + '.time']
            #if i > 1 and i < 4:
            #    t -= sd['L' + str(i - 1) + '.time']
            sumdata['L' + str(i) + '.time'].append(round(t, 1))
            sumdata['L' + str(i) + '.ratio'].append(
                round(100 - 100 * int(sd['L' + str(i) + '.acc']) / int(sd['L0.acc']), 1)
            )
            
#for k in sumdata:
#    print(k, len(sumdata[k]))
r = pd.DataFrame(data = sumdata)
r

NameError: name 'data' is not defined

In [24]:
def get_table(data):
    sumdata = {
        'tool': [],
        'autcount': [],
        'L0.acc': [],
    }
    for l in range(1, 5):
        for s in ['acc', 'time', 'ratio']:
            sumdata['L' + str(l) + '.' + s] = []

    for tool in automata_sets:
        data_df = data[tool]

        sumdata['tool'].append(tool)
        sd = data_df.sum()
        sumdata['autcount'].append(len(data_df))
        for i in range(0, 5):
            sumdata['L' + str(i) + '.acc'].append(int(sd['L' + str(i) + '.acc']))
            if i > 0:
                t = sd['L' + str(i) + '.time']
                #if i > 1 and i < 4:
                #    t -= sd['L' + str(i - 1) + '.time']
                sumdata['L' + str(i) + '.time'].append(round(t, 1))
                sumdata['L' + str(i) + '.ratio'].append(
                    round(100 - 100 * int(sd['L' + str(i) + '.acc']) / int(sd['L0.acc']), 1)
                )

    #for k in sumdata:
    #    print(k, len(sumdata[k]))
    r = pd.DataFrame(data = sumdata)
    return r

In [25]:
SCC_OPT_data = get_data("SCC_OPT")
SCC_UNOPT_data = get_data("SCC_UNOPT")
PREV_VERSION_data = get_data("PREV_VERSION")

`one_scc` =  <img src="https://i.upmath.me/svg/%20%5Cbigvee%5Climits_%7B%5Cdelta_%7BS_%7Bi%7D%7D%20%5Csubseteq%20%5Cdelta%7D%5Cbigleft(%20(%20%5Cbigvee%5Climits_%7Bt_%7Bi%7D%20%5Cin%20%5Cdelta_%7BS_%7Bi%7D%7D%7D%20e_i%20)%20%5Cland%20%20(%20%5Cbigwedge%5Climits_%7Bt_%7Bj%7D%20%5Cin%20%5Cdelta%20%5Csetminus%20%5Cdelta_%7BS_%7Bi%7D%7D%7D%20%5Clnot%20e_j%20%20)%20%5Cbigright)" alt=" \bigvee\limits_{\delta_{S_{i}} \subseteq \delta}\bigleft( ( \bigvee\limits_{t_{i} \in \delta_{S_{i}}} e_i ) \land  ( \bigwedge\limits_{t_{j} \in \delta \setminus \delta_{S_{i}}} \lnot e_j  ) \bigright)" />

`in_n_out` = <img src="https://i.upmath.me/svg/%5Cbigwedge%5Climits_%7Bq%20%5Cin%20Q%7D%20%5Cbigleft(%20(%20%5Cbigvee%5Climits_%7Bt_%7Bi%7D%20%5Cin%20%5Cdelta%20%5C%20%20%5Ccap%20%5C%20%5C%7B%20q%20%5C%7D%20%5Ctimes%202%5EM%20%5Ctimes%20%5CSigma%20%5Ctimes%20Q%7D%20e_i%20)%20%5CLongleftrightarrow(%20%5Cbigvee%5Climits_%7Bt_%7Bj%7D%20%5Cin%20%5Cdelta%20%5C%20%5Ccap%20%5C%20Q%20%5Ctimes%202%5EM%20%5Ctimes%20%5CSigma%20%5Ctimes%20%5C%7Bq%5C%7D%7D%20e_%7Bj%7D%20)%20%5Cbigright)" alt="\bigwedge\limits_{q \in Q} \bigleft( ( \bigvee\limits_{t_{i} \in \delta \  \cap \ \{ q \} \times 2^M \times \Sigma \times Q} e_i ) \Longleftrightarrow( \bigvee\limits_{t_{j} \in \delta \ \cap \ Q \times 2^M \times \Sigma \times \{q\}} e_{j} ) \bigright)" />



`true_cycles` = <img src="https://i.upmath.me/svg/%5Cbegin%7Balign*%7D%0A%20%20%5Cmcycle_3(%5Cvec%7Be%7D)%0A%20%20%26%3D%5Cbigvee_%7Bt%5Cin%5Cdelta%7De_t~%5Cwedge~%5Cforall%5Cvec%7Bs%7D~.~%5Czeta_%7B%5Cref%7Ben%3A1%7D%7D(%5Cvec%7Be%7D%2C%5Cvec%7Bs%7D)%5C%2C%5Cvee%5C%2C%0A%20%20%20%20%5Czeta_%7B%5Cref%7Ben%3A2%7D%7D(%5Cvec%7Be%7D%2C%5Cvec%7Bs%7D)%5C%2C%5Cvee%5C%2C%0A%20%20%20%20%5Czeta_%7B%5Cref%7Ben%3A3%7D%7D(%5Cvec%7Be%7D%2C%5Cvec%7Bs%7D)%5C%5C%20%25%5Cqquad%5Ctextrm%7Bwhere%7D%5C%5C%0A%20%20%5Czeta_%7B%5Cref%7Ben%3A1%7D%7D(%5Cvec%7Be%7D%2C%5Cvec%7Bs%7D)%20%26%3D%20%5Cbigwedge_%7Bt%3D(p%2Ca%2CM'%2Cq)%5Cin%5Cdelta%7D%5Chspace%7B-5mm%7D%5Cbig(e_t%5Cimplies%20(s_p%5Cwedge%20s_q)%5Cbig)%5C%5C%0A%20%20%5Czeta_%7B%5Cref%7Ben%3A2%7D%7D(%5Cvec%7Be%7D%2C%5Cvec%7Bs%7D)%20%26%3D%20%5Cbigwedge_%7Bt%3D(p%2Ca%2CM'%2Cq)%5Cin%5Cdelta%7D%5Chspace%7B-5mm%7D%5Cbig(e_t%5Cimplies%20(%5Cneg%20s_p%5Cwedge%20%5Cneg%20s_q)%5Cbig)%5C%5C%0A%20%20%5Czeta_%7B%5Cref%7Ben%3A3%7D%7D(%5Cvec%7Be%7D%2C%5Cvec%7Bs%7D)%20%26%3D%20%5Cbig(%5Cbigvee_%7Bt%3D(p%2Ca%2CM'%2Cq)%5Cin%5Cdelta%7D%5Chspace%7B-5mm%7De_t%5Cwedge%20s_p%5Cwedge%20%5Cneg%20s_q%5Cbig)%20~%5Cwedge~%0A%20%20%5Cbig(%5Cbigvee_%7Bt%3D(p%2Ca%2CM'%2Cq)%5Cin%5Cdelta%7D%5Chspace%7B-5mm%7De_t%5Cwedge%20%5Cneg%20s_p%5Cwedge%20s_q%5Cbig)%0A%5Cend%7Balign*%7D" alt="\begin{align*}
  \mcycle_3(\vec{e})
  &amp;=\bigvee_{t\in\delta}e_t~\wedge~\forall\vec{s}~.~\zeta_{\ref{en:1}}(\vec{e},\vec{s})\,\vee\,
    \zeta_{\ref{en:2}}(\vec{e},\vec{s})\,\vee\,
    \zeta_{\ref{en:3}}(\vec{e},\vec{s})\\ %\qquad\textrm{where}\\
  \zeta_{\ref{en:1}}(\vec{e},\vec{s}) &amp;= \bigwedge_{t=(p,a,M',q)\in\delta}\hspace{-5mm}\big(e_t\implies (s_p\wedge s_q)\big)\\
  \zeta_{\ref{en:2}}(\vec{e},\vec{s}) &amp;= \bigwedge_{t=(p,a,M',q)\in\delta}\hspace{-5mm}\big(e_t\implies (\neg s_p\wedge \neg s_q)\big)\\
  \zeta_{\ref{en:3}}(\vec{e},\vec{s}) &amp;= \big(\bigvee_{t=(p,a,M',q)\in\delta}\hspace{-5mm}e_t\wedge s_p\wedge \neg s_q\big) ~\wedge~
  \big(\bigvee_{t=(p,a,M',q)\in\delta}\hspace{-5mm}e_t\wedge \neg s_p\wedge s_q\big)
\end{align*}" />

`least_one_edge` = <img src="https://i.upmath.me/svg/%20%5Cbigvee_%7Be_t%20%5Cin%20S_i%7De_t" alt=" \bigvee_{e_t \in S_i}e_t" />

## Previous version

**L1**: `one_scc`

**L2**: `one_scc` ∧ `in_n_out`

**L3**: `true_cycle` ∧ `one_scc` ∧ `in_n_out`

In [27]:
t = get_table(PREV_VERSION_data)
t

,tool,autcount,L0.acc,L1.acc,L1.time,L1.ratio,L2.acc,L2.time,L2.ratio,L3.acc,L3.time,L3.ratio,L4.acc,L4.time,L4.ratio
0,delag,129,523,390,491.5,25.4,395,723.0,24.5,397,7998.4,24.1,386,9695.3,26.2
1,rabinizer4,234,882,566,559.5,35.8,569,690.0,35.5,567,904.8,35.7,543,1727.9,38.4
2,ltl3tela,91,348,332,309.3,4.6,334,360.6,4.0,327,422.6,6.0,323,948.1,7.2
3,ltl2tgba,70,198,198,587.3,0.0,198,612.6,0.0,189,858.2,4.5,189,914.9,4.5


## SCC optimized

**L1**: <img src="https://i.upmath.me/svg/%20%5Cbigwedge_%7BSCC%20%5C%3BS_i%7D%5Cbigvee_%7Be_t%20%5Cin%20S_i%7De_t" alt=" \bigwedge_{SCC \;S_i}\bigvee_{e_t \in S_i}e_t" />

**L2**: *for each SCC* `least_one_edge` ∧ `in_n_out`

**L3**: *for each SCC* `true_cycles`

In [28]:
t = get_table(SCC_OPT_data)
t

,tool,autcount,L0.acc,L1.acc,L1.time,L1.ratio,L2.acc,L2.time,L2.ratio,L3.acc,L3.time,L3.ratio,L4.acc,L4.time,L4.ratio
0,delag,129,523,390,503.5,25.4,395,711.8,24.5,398,2287.3,23.9,386,3314.3,26.2
1,rabinizer4,234,882,558,534.6,36.7,563,716.6,36.2,573,882.9,35.0,551,1635.8,37.5
2,ltl3tela,91,348,332,295.1,4.6,334,321.4,4.0,329,452.7,5.5,323,977.3,7.2
3,ltl2tgba,70,198,198,46.5,0.0,198,76.9,0.0,191,215.3,3.5,191,272.9,3.5


**L1**: `least_one_edge`

**L2**: `least_one_edge` ∧ in_n_out

**L3**: true_cycles

In [29]:
t = get_table(SCC_UNOPT_data)
t

,tool,autcount,L0.acc,L1.acc,L1.time,L1.ratio,L2.acc,L2.time,L2.ratio,L3.acc,L3.time,L3.ratio,L4.acc,L4.time,L4.ratio
0,delag,129,523,400,550.4,23.5,407,756.9,22.2,422,8344.5,19.3,387,9963.4,26.0
1,rabinizer4,234,882,553,573.5,37.3,570,744.4,35.4,589,952.5,33.2,546,1730.1,38.1
2,ltl3tela,91,348,339,312.5,2.6,339,352.5,2.6,333,583.9,4.3,332,1126.6,4.6
3,ltl2tgba,70,198,198,57.7,0.0,198,106.3,0.0,191,539.5,3.5,191,568.8,3.5


U ltl3tela bylo na vstupu 89 -- to se děje z důvodu, že na konci `telatka` mam zapnutý `equivalence check` (ze spot knihovny), zjednodusovani trvalo tak dlouho, ze muselo byt zabite signalem 9 (SIGKILL 9 Term Kill signal), po odstraneni `equivalence check` dobehne okamzite.  
Possible report na spot.